In [1]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
import tqdm

### In this model, due to the limited resources I had, I wasn’t able to provide the model with all the necessary data, nor could I train it for the required number of epochs. But since the structure is correct, it’s easy to run this process anyway.

In [2]:
text='''I am Harry Potter. I am a young boy who lives with my aunt and uncle, the Dursleys, in a small house on Privet Drive. I am curious about the world and I often wonder why I feel different from others. One day, I receive a letter from Hogwarts School of Witchcraft and Wizardry. I am surprised and excited because I learn that I am a wizard. I have never seen magic before, and I am eager to discover all the wonders that await me. I pack my things and travel to Hogwarts, feeling nervous and thrilled at the same time. I meet many students on the train and I make friends with Ron Weasley and Hermione Granger. I am happy to have friends who understand me and share my excitement. We explore the castle together and discover hidden rooms, secret passages, and magical creatures. I see dragons, hippogriffs, house-elves, unicorns, and other creatures I have never imagined. I learn many spells and practice them every day. I try simple charms, defensive spells, and more powerful magic. I am careful when using magic because I want to do things correctly and safely. I attend many classes at Hogwarts, including Potions, Defense Against the Dark Arts, Transfiguration, Herbology, and Flying Lessons. I play Quidditch on a broomstick and practice teamwork with my friends. I learn strategy, courage, and cooperation. I face many challenges and adventures. I discover the secret of the Philosopher’s Stone and protect it from those who want to steal it. I fight trolls, solve puzzles, and confront dark wizards. I learn that courage, loyalty, and friendship are more important than any spell. I help my friends when they are in danger and I protect those who need help. I practice magic every day and read books to improve my knowledge. I observe magical creatures and learn how to care for them. I explore the castle and the grounds, discovering new places and hidden secrets. I meet ghosts, magical plants, and enchanted objects. I try to understand their powers and how to use them responsibly. I grow stronger and wiser with every year at Hogwarts. I learn about the history of magic and the ongoing fight between good and evil. I discover that love and loyalty are powerful forms of magic that cannot be defeated. I practice spells, study lessons, and learn life lessons. I make mistakes, but I try to learn from them. I act with courage even when situations are dangerous or uncertain. I share my knowledge with friends and help them overcome challenges. I remember the advice of my teachers and follow guidance when necessary. I explore secret corridors, hidden rooms, and mysterious passages. I encounter magical creatures that teach me new things about the world. I care for my friends and help them when they are in trouble. I feel proud when we succeed together and celebrate small victories. I understand that honesty, bravery, and kindness are stronger than any magic spell. I practice patience and persistence, even when learning is difficult. I know that helping others is more important than personal gain. I continue to explore Hogwarts and the magical world. I meet new students, learn new spells, and discover magical plants and objects. I face challenges that test my courage, intelligence, and compassion. I try to be responsible and protect others whenever possible. I grow more confident and wise each year. I think before I act and trust my instincts. I understand that mistakes are lessons and challenges are opportunities. I feel proud of my achievements and grateful for my friends and teachers. I continue to practice magic, learn new spells, and explore the magical world. I believe that even ordinary people can do extraordinary things if they believe in themselves and support others. I am Harry Potter, a young wizard learning about magic, friendship, courage, love, and responsibility. I am ready to face new adventures, help my friends, and protect the magical world. I am determined to be brave, kind, and wise. I am learning, exploring, and growing every day. I discover new things about magic, about people, and about myself. I face dangers, solve mysteries, and meet magical creatures. I practice spells, play Quidditch, and attend lessons. I learn teamwork, responsibility, and courage. I help friends, protect Hogwarts, and try to do what is right. I continue to discover secrets of the castle, learn new magic, and improve my skills. I face dark forces and learn how to fight them. I understand that courage, hope, and friendship are more powerful than fear. I act with kindness, loyalty, and bravery in every situation. I continue to grow as a wizard and as a person. I explore, learn, and practice magic every day. I believe in myself and in the power of my friends. I know that together we can overcome obstacles and achieve extraordinary things. I am ready for new challenges, new lessons, and new adventures. I continue to write my story at Hogwarts, learning from each day, and sharing experiences with friends. I discover the importance of love, courage, and perseverance. I help others when they need me and I protect those who cannot protect themselves. I practice magic responsibly, learn from mistakes, and grow stronger every day. I am Harry Potter, a wizard who is learning about life, magic, friendship, and courage. I am ready to face the world with bravery, wisdom, and kindness. I continue to learn, explore, and grow, discovering the magical world and my place in it.'''

# Free runing

In this model, we use the model’s own output — meaning the word that the model itself generates — as the input for the next layer. This mechanism can be seen as a form of learning from its own predictions and can be converted for training purposes. However, in the real world, it is rarely used and serves a different purpose 😅.

In [54]:
embeded_size=20

In [93]:
class rnn_free_runing(nn.Module):
    def __init__(self,text,embeded_size):
        super().__init__()
        #defining variables
        self.text=text
        self.words=text.split()
        self.vocab=list(set(self.words))
        self.embeded_size=embeded_size

        # defining weights of words
        self.embeding=nn.Embedding(embedding_dim=embeded_size,num_embeddings=len(self.vocab))
        # defin weight of network

        #input weight
        self.whx = nn.Parameter(torch.randn(embeded_size, embeded_size))
        #hidden state weight
        self.whh = nn.Parameter(torch.randn(embeded_size,embeded_size))
        #hidden-to-output weight
        self.why = nn.Parameter(torch.randn(embeded_size,len(self.vocab)))
        #hidden state bias
        self.bh=nn.Parameter(torch.randn(embeded_size))
        #bias of output
        self.by=nn.Parameter(torch.randn(len(self.vocab)))

        self.relu=nn.ReLU()




    def forward(self,idx,ht):


        #get weight of start word
        embeded=self.embeding(idx)


        at=(embeded @ self.whx) + (ht @ self.whh)+self.bh
        ht=self.relu(at)

        output=self.by + (ht @ self.why)

        return output,ht




In [94]:
#splited text
words=text.split()

vocab=list(set(words))

words_idx={word:i for i,word in enumerate(vocab)}
idx_word={i:word for i,word in enumerate(vocab)}

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#objected model
model_free_runing=rnn_free_runing(text=text,embeded_size=embeded_size).to(device)
#adam optimizer
optim=torch.optim.Adam(model_free_runing.parameters(),lr=1e-4)

loss_fn=nn.CrossEntropyLoss()

#get idx of all words of text
idxes=[words_idx[word] for i,word in enumerate(words)]
#convert to tensor
idxes=torch.tensor(idxes)

Because cross-entropy itself applies softmax and log on its input, I didn’t apply argmax on the predictions.

In [96]:
train_loss=[]
#start to training
model_free_runing.train()
ht=torch.zeros(embeded_size,device=device)



start_word=words[0]
#get idx of start word
start_idx=words_idx[start_word]
#conver to tensor
idx=torch.tensor(start_idx)




for epochs in tqdm.trange(10):
  for i in range(len(words)):

        data=idxes[i].to(device).long()
        optim.zero_grad()
        predict,ht=model_free_runing.forward(idx,ht=ht)

        #I didn’t apply softmax, log, or argmax because
        #the CrossEntropy function calculates all of these on the logits



        loss=loss_fn(predict,data)


        idx=torch.argmax(predict)

        loss.backward()
        optim.step()
        ht=ht.detach()

        train_loss.append(loss.item())

100%|██████████| 10/10 [00:14<00:00,  1.47s/it]


In [58]:
with torch.no_grad():
  for i in range(10):

    logits = model_free_runing.forward(idx,ht)
    idx=torch.argmax(logits)

    word=[idx_word[int(idx)]]
    print(word)

['friends,']
['Hermione']
['creatures.']
['house']
['confident']
['wonders']
['situations']
['broomstick']
['friends,']
['Hermione']


# Teacher Forsing

In this model, unlike the free-running algorithm, we don’t use the model’s own output as the input for the next time step. Instead, we use the correct word as input. This structure is more practical in industry, easier to train, and better covers the intended task. Of course, it is also more structured and logical.

In [73]:
class teacher_forecing_rnn(nn.Module):
    def __init__(self,text,embeded_size):
        super().__init__()
        #defining variables
        self.text=text
        self.words=text.split()
        self.vocab=list(set(self.words))
        self.embeded_size=embeded_size

        # defining weights of words
        self.embeding=nn.Embedding(embedding_dim=embeded_size,num_embeddings=len(self.vocab))
        # defin weight of network

        #input weight
        self.whx = nn.Parameter(torch.randn(embeded_size, embeded_size))
        #hidden state weight
        self.whh = nn.Parameter(torch.randn(embeded_size,embeded_size))
        #hidden-to-output weight
        self.why = nn.Parameter(torch.randn(embeded_size,len(self.vocab)))
        #hidden state bias
        self.bh=nn.Parameter(torch.randn(embeded_size))
        #bias of output
        self.by=nn.Parameter(torch.randn(len(self.vocab)))

        self.relu=nn.ReLU()




    def forward(self,idx,ht):





        #get weight of start word
        embeded=self.embeding(idx)



        at=(embeded @ self.whx) + (ht @ self.whh)+self.bh
        ht=self.relu(at)

        output=self.by + (ht @ self.why)


        return output,ht




Because cross-entropy itself applies softmax and log on its input, I didn’t apply argmax on the predictions.

In [74]:


words=text.split()

vocab=list(set(words))
words_idx={word:i for i,word in enumerate(vocab)}
idx_word={i:word for i,word in enumerate(vocab)}

#device

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_teacher_forecing=teacher_forecing_rnn(text=text,embeded_size=embeded_size).to(device)

#adam optimizer
optim=torch.optim.Adam(model_teacher_forecing.parameters(),lr=1e-4)

loss_fn=nn.CrossEntropyLoss()

idxes=[words_idx[word] for i,word in enumerate(words)]
idxes=torch.tensor(idxes)

In [97]:
train_loss=[]
#start to training
model_teacher_forecing.train()
ht=torch.zeros(embeded_size,device=device)








for epochs in tqdm.trange(10):
  for i in range(len(words)-1):

        idx=idxes[i].to(device).long()
        optim.zero_grad()
        predict,ht=model_teacher_forecing.forward(data,ht)

        #I didn’t apply softmax, log, or argmax because
        #the CrossEntropy function calculates all of these on the logits

        data=idxes[i+1].to(device).long()



        loss=loss_fn(predict,data)


        #idx=torch.argmax(predict)

        loss.backward()
        optim.step()

        ht=ht.detach()

        train_loss.append(loss.item())

100%|██████████| 10/10 [00:07<00:00,  1.29it/s]


In [99]:
start_word=words[0]
#get idx of start word
start_idx=words_idx[start_word]
#conver to tensor
idx=torch.tensor(start_idx)

In [100]:
with torch.no_grad():
  for i in range(10):
    logits,ht = model_teacher_forecing.forward(idx,ht)
    idx=torch.argmax(logits)

    print(idx_word[int(idx)])





School
School
School
School
School
School
School
School
School
School


From a structural perspective, as I mentioned before, it is completely clear that teacher forcing is better than free-running.